# Przypisanie sentymentu dla jak największej liczby słów

1. Import danych z projektu NAWL - Nencki Affective Word List z oznaczeniem sentymentu
2. Przypisanie sentymentu w oparciu o podstawowe słowa
3. Wykorzystanie embedingów do przypisania sentymentu na podstawie sąsiadujących słów

In [32]:
import re
import pandas as pd
import numpy as np
import funcs 
import importlib
import unicodecsv as csv
import pickle
import random
import gensim
from gensim import corpora, models
from timeit import default_timer as timer
from datetime import datetime, timedelta
import nltk
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
import spacy
from collections import Counter
import pyLDAvis
import pyLDAvis.gensim
from sklearn.model_selection import train_test_split
import scipy

importlib.reload(funcs)

<module 'funcs' from 'D:\\_projekty\\praca\\funcs.py'>

In [5]:
%time dane = pd.read_csv('00_dane/dane_cleanest.csv',  sep='|', encoding='UTF-8')
%time dane['speech_rawest'] = dane['speech_rawest'].apply(lambda x: re.sub(r'[\W\d]',' ', x))
%time dane['speech_rawest'] = dane['speech_rawest'].apply(lambda x: [word for word in x.split()])

Wall time: 14.6 s
Wall time: 33.9 s
Wall time: 5.99 s


In [33]:
dane.tail()

,nr,date_,speech_rawer,full_name,party_final,speech_rawest,topic_LDA_BOW,topic_LDA_TF,opis_LDA_BOW,opis_LDA_TF
158315,158874,31-08-2011,Sojuszowi Lewicy Demokratycznej bardzo zależy...,Romuald Ajchler,Sojusz Lewicy Demokratycznej,"[sojusz, lewica, demokratyczny, zależeć, ziemi...",26.0,8.0,posiedzenie,posiedzenie
158316,158875,31-08-2011,Panie Przewodniczący Krajowej Rady Sądownictw...,Stanisław Rydzoń,Sojusz Lewicy Demokratycznej,"[przewodniczący, krajowy, rada, sądownictwo, z...",16.0,13.0,wymiar sprawiedliwości,przepisy
158317,158876,31-08-2011,Mam pytanie do pana ministra. Panie ministrze...,Stanisław Rydzoń,Sojusz Lewicy Demokratycznej,"[mieć, pytanie, minister, minister, mieć, doty...",28.0,2.0,handel,inwestycje
158318,158877,31-08-2011,Pani Minister! Bardzo dużo mówi się o szkolni...,Stanisław Stec,Sojusz Lewicy Demokratycznej,"[minister, mówić, szkolnictwo, zawodowy, dosto...",7.0,2.0,NaN,inwestycje
158319,158878,31-08-2011,Szanowna Pani Minister! Bardzo ważnym zadanie...,Tadeusz Tomaszewski,Sojusz Lewicy Demokratycznej,"[szanowny, minister, ważny, zadanie, władza, p...",25.0,11.0,samorządy,rodzina


## 1. Wczytanie listy słów z NAWL

Parametry wg których przypisano kategorie zmieniono aby zwiększyć liczbę przypisań. Usunięto grupę 'neutral'.

In [109]:
NAWL = pd.read_csv('00_dane/nawl-analysis_2.csv', encoding='UTF-8')
NAWL.head()

,word,category,mean Happiness,mean Anger,mean Sadness,mean Fear,mean Disgust,distance to H,distance to A,distance to S,distance to F,distance to D,distance to N
0,tropiki,H,4.518518,1.703704,1.592593,2.074074,1.629630,3.637347,7.251989,7.312897,6.880706,7.236647,4.675840
1,łączyć,H,3.814815,1.666667,1.925926,1.777778,1.666667,4.161421,6.728334,6.476909,6.615627,6.693552,3.628548
2,bogaty,H,4.592593,2.370370,1.703704,1.703704,1.925926,3.826875,6.801012,7.372475,7.404916,7.217664,4.897808
3,rozkoszny,H,5.296296,1.407407,1.481481,1.555556,1.370370,2.366185,7.513857,7.449380,7.387380,7.536883,4.835511
4,nabytek,H,4.555555,1.666667,1.666667,1.555556,1.444444,3.227771,7.022504,6.998906,7.096976,7.198279,4.325238


In [102]:
print('Lista zawiera {} słów.'.format(len(NAWL)))

Lista zawiera 2902 słów.


Analiza wczytanych danych

In [7]:
NAWL.groupby('category')["word"].count()
#NAWL.groupby('category')["mean Anger"].mean()

category
A     141
D      59
F     216
H    1193
S      93
U    1200
Name: word, dtype: int64

Klasa "U" oznacza brak klasyfikacji. Mamy więc mało przypisanych słów :/

Inne:
- Happiness
- Anger
- Sadness
- Fear
- Neutral (zmia=eniono parametry tak, aby się nie dopsiywało)
- Disgust


Przypiszmy mimo wszystko flagę, do której odległość jest najmniejsza.

In [85]:
a = NAWL.iloc[2]
print(a)

word              przyjaciel
category                   H
mean Happiness       6.03704
mean Anger           1.44444
mean Sadness         1.62963
mean Fear            1.55556
mean Disgust         1.03704
distance to H        1.90164
distance to A          7.839
distance to S        7.69145
distance to F        7.75581
distance to D        8.13595
distance to N        5.40917
Name: 2, dtype: object


In [86]:
max(a['distance to H'], a['distance to A'], a['distance to S'], a['distance to F'], a['distance to D'], a['distance to N'])

8.135946

In [110]:
NAWL['new_flag']=''
for i in range(len(NAWL)):
    a = NAWL.iloc[i]
    m = min(a['distance to H'], a['distance to A'], a['distance to S'], a['distance to F'], a['distance to D'])
    if m == a['distance to H']:
        NAWL['new_flag'].iloc[i]='H'
    elif m == a['distance to S']:
        NAWL['new_flag'].iloc[i]='S'
    elif m == a['distance to F']:
        NAWL['new_flag'].iloc[i]='F'
    elif m == a['distance to D']:
        NAWL['new_flag'].iloc[i]='D'
    elif m == a['distance to A']:
        NAWL['new_flag'].iloc[i]='A'

C:\Users\Magda\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [111]:
NAWL.groupby('new_flag')["word"].count()

new_flag
A     322
D     149
F     465
H    1740
S     226
Name: word, dtype: int64

In [103]:
NAWL[(NAWL['category']!=NAWL['new_flag']) & (NAWL['category']!='U')]

,word,category,mean Happiness,mean Anger,mean Sadness,mean Fear,mean Disgust,distance to H,distance to A,distance to S,distance to F,distance to D,distance to N,new_flag


In [112]:
NAWL.drop('distance to N', axis=1)

,word,category,mean Happiness,mean Anger,mean Sadness,mean Fear,mean Disgust,distance to H,distance to A,distance to S,distance to F,distance to D,new_flag
0,tropiki,H,4.518518,1.703704,1.592593,2.074074,1.629630,3.637347,7.251989,7.312897,6.880706,7.236647,H
1,łączyć,H,3.814815,1.666667,1.925926,1.777778,1.666667,4.161421,6.728334,6.476909,6.615627,6.693552,H
2,bogaty,H,4.592593,2.370370,1.703704,1.703704,1.925926,3.826875,6.801012,7.372475,7.404916,7.217664,H
3,rozkoszny,H,5.296296,1.407407,1.481481,1.555556,1.370370,2.366185,7.513857,7.449380,7.387380,7.536883,H
4,nabytek,H,4.555555,1.666667,1.666667,1.555556,1.444444,3.227771,7.022504,6.998906,7.096976,7.198279,H
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2897,guz,U,1.148148,3.666667,4.703704,4.666666,3.703704,9.322883,7.693601,6.701199,6.865419,7.687548,S
2898,asekuracyjny,U,3.259259,1.592593,1.629630,2.925926,1.666667,5.050983,6.919656,6.853235,5.591976,6.836249,H
2899,ambona,U,2.074074,2.444444,1.925926,2.148148,2.296296,6.417684,6.330169,6.712734,6.486877,6.465244,A
2900,bezsensowny,U,1.740741,3.444444,3.037037,2.666667,2.407408,7.320939,6.228213,6.572936,7.014468,7.238194,A


## 2. Przypisanie sentymentu słów

### 2.1 Budowa słownika wszystkich słów

In [5]:
dict_sp = {}
for sp in dane['speech_rawest']:
    for w in sp:
        if w not in dict_sp.keys():
            dict_sp[w]=0
        dict_sp[w]+=1

In [9]:
print('Liczba unikalnych słów w słowniku wynosi {}.'.format(len(dict_sp)))

Liczba unikalnych słów w słowniku wynosi 152498.


In [111]:
df_sp = pd.DataFrame(list(dict_sp.items()), columns=['word', 'counter'])
df_sp.head()

,word,counter
0,chcieć,205353
1,zdanie,46410
2,powiedzieć,114946
3,temat,36539
4,skutek,31434


In [20]:
len(df_sp)

152498

### 2.2 Dogranie parametrów

In [112]:
df_sp = pd.merge(df_sp, NAWL, left_on='word', right_on='word', how='left' )

In [113]:
df_sp[200:210]

,word,counter,category,mean Happiness,mean Anger,mean Sadness,mean Fear,mean Disgust,distance to H,distance to A,distance to S,distance to F,distance to D,distance to N,new_flag
200,placówka,5440,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
201,zagraniczny,27255,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
202,wrócić,5807,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
203,kraj,81373,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
204,śmierć,7324,U,1.296296,3.814815,5.888889,6.000000,3.148148,10.201947,8.788602,7.118899,7.020968,9.272644,8.381661,F
205,odmówić,1955,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
206,sprawie,42793,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
207,uważać,48841,U,2.153846,1.923077,1.423077,2.500000,1.307692,5.717540,6.015101,6.538665,5.321966,6.624956,2.721546,F
208,kierować,19754,H,4.500000,1.692308,1.653846,2.576923,1.307692,3.921665,7.148361,7.184477,6.268880,7.454568,4.439679,H
209,przesłanka,7073,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Ile mamy dogranych?

In [124]:
print("Uzupełniony sentyment dla {} z {} słów.".format(len(df_sp[pd.isnull(df_sp['new_flag'])==False]), len(df_sp)))

Uzupełniony sentyment dla 2696 z 152498 słów.


In [115]:
pickle.dump(df_sp, open('df_sp.p', 'wb'))

## 3. Dogranie embeddingów

In [24]:
df_sp = pickle.load(open('df_sp.p','rb'))

In [25]:
df_sp.head()

,word,counter,category,mean Happiness,mean Anger,mean Sadness,mean Fear,mean Disgust,distance to H,distance to A,distance to S,distance to F,distance to D,distance to N,new_flag
0,chcieć,205353,H,4.333334,1.777778,2.037037,2.148148,1.592593,4.030499,7.033235,6.847620,6.729651,7.201385,4.369833,H
1,zdanie,46410,H,2.962963,1.222222,1.185185,1.444444,1.148148,4.453825,6.560185,6.583048,6.347609,6.579155,2.303450,H
2,powiedzieć,114946,U,3.000000,1.346154,1.538462,2.307692,1.192308,5.129413,6.853925,6.679800,5.946053,6.967851,2.799579,H
3,temat,36539,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,skutek,31434,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
import fasttext.util
fasttext.util.download_model('pl', if_exists='ignore')  # English


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 (100.00%) [==================================================>]================================================> ]=================================================> ]> ]=================================================> ]=================================================> ]=================================================> ]=================================================> ]=================================================> ]=================================================> ]=================================================> ]=================================================> ]


'cc.pl.300.bin'

In [12]:
ft = fasttext.load_model('cc.pl.300.bin')

In [13]:
ft.get_word_vector('sejm').shape


(300,)

In [21]:
%time ft.get_nearest_neighbors('polityka')

Wall time: 599 ms


[(0.7485365867614746, 'Polityka'),
 (0.6835419535636902, 'polityka.'),
 (0.6773313879966736, 'strategia'),
 (0.6700135469436646, '-Polityka'),
 (0.6477417945861816, 'lityka'),
 (0.6407139897346497, 'geopolityka'),
 (0.6372516751289368, 'dyplomacja'),
 (0.6315040588378906, 'biopolityka'),
 (0.6287774443626404, 'serwisupolityka'),
 (0.627938985824585, 'krytyka')]

In [17]:
dir(ft)

['__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_labels',
 '_words',
 'f',
 'get_analogies',
 'get_dimension',
 'get_input_matrix',
 'get_input_vector',
 'get_label_id',
 'get_labels',
 'get_line',
 'get_meter',
 'get_nearest_neighbors',
 'get_output_matrix',
 'get_sentence_vector',
 'get_subword_id',
 'get_subwords',
 'get_word_id',
 'get_word_vector',
 'get_words',
 'is_quantized',
 'labels',
 'predict',
 'quantize',
 'save_model',
 'set_args',
 'set_matrices',
 'test',
 'test_label',
 'words']

In [19]:
ft.get_nearest_neighbors("dom", k=20)

[(0.7359972596168518, 'domek'),
 (0.7010008692741394, 'Dom'),
 (0.648328959941864, 'budynek'),
 (0.6448397636413574, 'ogród'),
 (0.6375203728675842, 'dworek'),
 (0.6291686296463013, 'mieszkanie'),
 (0.6271489262580872, 'domy'),
 (0.6179277300834656, 'jednorodzinny'),
 (0.6130203008651733, 'dom-'),
 (0.6055596470832825, 'mieszkalny'),
 (0.5954145193099976, 'samochód'),
 (0.5952804088592529, 'domek-'),
 (0.5934181213378906, 'domek.'),
 (0.5884250402450562, 'rodzinny'),
 (0.5861175656318665, 'parterowy'),
 (0.5834079384803772, 'pałac'),
 (0.5712330341339111, 'kościół'),
 (0.5673616528511047, 'dwór'),
 (0.5642721652984619, 'przytulny'),
 (0.5632957220077515, 'domeczek')]

In [27]:
df_sp[pd.isnull(df_sp['new_flag'])==True]

,word,counter,category,mean Happiness,mean Anger,mean Sadness,mean Fear,mean Disgust,distance to H,distance to A,distance to S,distance to F,distance to D,distance to N,new_flag
3,temat,36539,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,skutek,31434,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,podjęcie,10125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,zasadny,6345,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,konieczny,15106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152493,kowalczys,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
152494,puczyński,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
152495,ikrzyć,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
152496,unwin,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
v1 = ft.get_word_vector("tata")
v2 = ft.get_word_vector("mama")
v3 = ft.get_word_vector("wujek")
v4 = ft.get_word_vector("kot")

In [45]:
ft.get_analogies("tata", "wujek", "mama")

[(0.7079054117202759, 'Mama'),
 (0.6649925708770752, 'babcia'),
 (0.6591989398002625, 'mamusia'),
 (0.6386964321136475, 'mama-'),
 (0.634793758392334, 'matka'),
 (0.6302740573883057, 'córcia'),
 (0.6218123435974121, '-mama'),
 (0.6194969415664673, 'córeczka'),
 (0.6155368685722351, 'córka'),
 (0.6099972128868103, 'mama.')]

In [37]:
1-scipy.spatial.distance.cosine(ft.get_word_vector("dom"), ft.get_word_vector("domek"))

0.7359974980354309

In [38]:
NAWL

,word,category,mean Happiness,mean Anger,mean Sadness,mean Fear,mean Disgust,distance to H,distance to A,distance to S,distance to F,distance to D,distance to N,new_flag
0,tropiki,H,4.518518,1.703704,1.592593,2.074074,1.629630,3.637347,7.251989,7.312897,6.880706,7.236647,4.675840,H
1,łączyć,H,3.814815,1.666667,1.925926,1.777778,1.666667,4.161421,6.728334,6.476909,6.615627,6.693552,3.628548,H
2,bogaty,H,4.592593,2.370370,1.703704,1.703704,1.925926,3.826875,6.801012,7.372475,7.404916,7.217664,4.897808,H
3,rozkoszny,H,5.296296,1.407407,1.481481,1.555556,1.370370,2.366185,7.513857,7.449380,7.387380,7.536883,4.835511,H
4,nabytek,H,4.555555,1.666667,1.666667,1.555556,1.444444,3.227771,7.022504,6.998906,7.096976,7.198279,4.325238,H
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2897,guz,U,1.148148,3.666667,4.703704,4.666666,3.703704,9.322883,7.693601,6.701199,6.865419,7.687548,6.816577,S
2898,asekuracyjny,U,3.259259,1.592593,1.629630,2.925926,1.666667,5.050983,6.919656,6.853235,5.591976,6.836249,3.789807,H
2899,ambona,U,2.074074,2.444444,1.925926,2.148148,2.296296,6.417684,6.330169,6.712734,6.486877,6.465244,3.296023,A
2900,bezsensowny,U,1.740741,3.444444,3.037037,2.666667,2.407408,7.320939,6.228213,6.572936,7.014468,7.238194,4.791221,A


In [41]:
NAWL_vec_dict = {}
for w in NAWL["word"]:
    NAWL_vec_dict[w] = ft.get_word_vector(w)

In [46]:
len(NAWL_vec_dict)

2902

In [47]:
pickle.dump(NAWL_vec_dict, open('nawl.p', 'wb'))

In [ ]:
for word in df_sp['word']

In [48]:
corp_vec_dict = {}
for w in df_sp["word"]:
    corp_vec_dict[w] = ft.get_word_vector(w)

In [130]:
word='sprawie'

word_vec = corp_vec_dict[word]
word_dict = {}
neigh_df = pd.DataFrame(columns=['distance to H','distance to A','distance to S','distance to F','distance to D'])
for k in NAWL_vec_dict.keys():
    word_dict[k] = scipy.spatial.distance.cosine(word_vec, NAWL_vec_dict[k])

for neighbor, dist in sorted(word_dict.items(), key=lambda item: item[1])[:neigh_cnt]:
    if dist<=0.9:
        neigh_df = neigh_df.append(NAWL[['distance to H','distance to A','distance to S','distance to F','distance to D']][NAWL['word']==neighbor] )

if len(neigh_df>0):
    df_sp.loc[df_sp.word==word, 'distance to H'] = neigh_df.mean()['distance to H']
    df_sp.loc[df_sp.word==word, 'distance to A'] = neigh_df.mean()['distance to A']
    df_sp.loc[df_sp.word==word, 'distance to S'] = neigh_df.mean()['distance to S']
    df_sp.loc[df_sp.word==word, 'distance to F'] = neigh_df.mean()['distance to F']
    df_sp.loc[df_sp.word==word, 'distance to D'] = neigh_df.mean()['distance to D']

    m = neigh_df.mean().min()
    if m == neigh_df.mean()['distance to H']:
        df_sp.loc[df_sp.word==word, 'new_flag']='H'
    elif m == neigh_df.mean()['distance to S']:
        df_sp.loc[df_sp.word==word, 'new_flag']='S'
    elif m == neigh_df.mean()['distance to F']:
        df_sp.loc[df_sp.word==word, 'new_flag']='F'
    elif m == neigh_df.mean()['distance to D']:
        df_sp.loc[df_sp.word==word, 'new_flag']='D'
    elif m == neigh_df.mean()['distance to A']:
        df_sp.loc[df_sp.word==word, 'new_flag']='A'

In [156]:
word='sprawie'

word_vec = corp_vec_dict[word]
word_dict = {}
neigh_df = pd.DataFrame(columns=['distance to H','distance to A','distance to S','distance to F','distance to D'])
for k in NAWL_vec_dict.keys():
    word_dict[k] = scipy.spatial.distance.cosine(word_vec, NAWL_vec_dict[k])

In [158]:
for neighbor, dist in sorted(word_dict.items(), key=lambda item: item[1])[:neigh_cnt]:
    if dist<=0.5:
        neigh_df = neigh_df.append(NAWL[['distance to H','distance to A','distance to S','distance to F','distance to D']][NAWL['word']==neighbor] )


In [159]:
len(neigh_df)

0

In [160]:
aaa = pd.Series(list(word_dict.values()))
aaa.describe()

count    2902.000000
mean        0.905250
std         0.067914
min         0.581126
25%         0.864111
50%         0.913242
75%         0.951734
max         1.085228
dtype: float64

In [145]:
ne-

{'tropiki': 0.972013808786869,
 'łączyć': 0.8906546756625175,
 'bogaty': 0.9376760423183441,
 'rozkoszny': 0.9839063975960016,
 'nabytek': 0.9334661439061165,
 'matowy': 0.8962822407484055,
 'przebywać': 0.8802626579999924,
 'tolerancja': 0.7790106236934662,
 'ludzie': 0.8795018941164017,
 'Japonia': 0.89582309871912,
 'rozsądny': 0.8625612258911133,
 'ożywiony': 0.9086886793375015,
 'motorower': 0.9080251902341843,
 'liczba': 0.6790712773799896,
 'kufer': 0.9813745710998774,
 'buk': 0.974748507142067,
 'grać': 0.9077311754226685,
 'odwaga': 0.6950816214084625,
 'świerk': 0.9487426355481148,
 'wino': 0.9728258233517408,
 'bóbr': 1.0265690479427576,
 'zatwierdzać': 0.8859542906284332,
 'spojrzenie': 0.8877359703183174,
 'nazywać': 0.9187918081879616,
 'spadek': 0.8611609488725662,
 'brat': 0.9310429394245148,
 'świetny': 0.8836479261517525,
 'honor': 0.8551069796085358,
 'marzyć': 0.8608828783035278,
 'witalny': 0.9361825585365295,
 'symfonia': 0.7693488150835037,
 'brytyjski': 0.975981

In [131]:
print("Uzupełniony sentyment dla {} z {} słów.".format(len(df_sp[pd.isnull(df_sp['new_flag'])==False]), len(df_sp)))

Uzupełniony sentyment dla 2696 z 152498 słów.


In [129]:

    print(word)

chcieć
zdanie
powiedzieć
ustawa
inicjatywa
należeć
historia
wnioskować
komisja
stan
nielegalny
myśleć
oko
człowiek
cierpieć
sędzia
wyrok
obowiązywać
mówić
orzeczenie
wydawać
kara
czyn
więzienie
podawać
państwo
rzeczywistość
móc
sytuacja
olbrzymi
znać
przestępstwo
granica
sąd
praca
wypadek
rzecz
wysoki
izba
pobyt
powstanie
stać
straż
wykonywać
podejmować
minister
dotyczyć
oficer
placówka
kraj
śmierć
uważać
kierować
agent
niesprawiedliwy
krzywdzić
ojczyzna
musieć
obserwować
następca
prosić
słowo
istnieć
prawdziwy
niezgodny
uczestniczyć
sens
punkt
zakres
zbrodnia
życie
zdrowie
stosować
wolność
proponować
tytuł
stosunek
trwać
debata
wrażenie
sojusz
demokratyczny
uzyskać
miejsce
rozumieć
wynikać
zły
ocena
fakt
kartka
stopień
rozwiązanie
rachunek
partia
potrzebować
twierdzić
droga
demokracja
wymagać
spokojny
początek
towarzyszyć
dawać
rozmawiać
propozycja
słabość
moralny
zrozumiały
zwycięzca
strona
winny
łamać
cierpienie
konflikt
minuta
przyszłość
siła
szukać
rozwiązywać
problem
działać
widz

In [133]:
word

'zatroskany'

In [132]:
sorted(word_dict.items(), key=lambda item: item[1])[:neigh_cnt]

[('zatroskany', 0.0),
 ('zmartwiony', 0.3334028720855713),
 ('zdziwiony', 0.4051198959350586),
 ('przerażony', 0.44494062662124634),
 ('przygnębiony', 0.4686475992202759)]

In [117]:
neigh_df

,distance to H,distance to A,distance to S,distance to F,distance to D
2793,6.515241,6.189839,6.500107,5.635338,6.244465
1764,5.618850,6.296969,6.785876,6.181795,7.027832
822,5.308498,6.833667,7.275909,6.528132,7.046259
1441,7.029672,7.586001,6.423764,5.823464,7.221201
2754,6.772903,6.728729,6.621348,6.462495,6.562424


In [88]:
df_sp['distance to H'][df_sp['word']==word].value = neigh_df.mean()['distance to H']
#df_sp[['distance to H','distance to A','distance to S','distance to F','distance to D']][df_sp['word']==word] = neigh_df.mean()

In [121]:
df_sp[df_sp['word']==word]

,word,counter,category,mean Happiness,mean Anger,mean Sadness,mean Fear,mean Disgust,distance to H,distance to A,distance to S,distance to F,distance to D,distance to N,new_flag
203,kraj,81373,NaN,NaN,NaN,NaN,NaN,NaN,4.201303,7.211103,6.839162,6.865976,7.479072,NaN,H
